In [1]:
#!g1.1
from config import config
import pickle
import json

with open(config['dataset']['postprocess']['train'], 'r') as f:
    el_single_results = json.load(f)
# with open(config['wikidata']['transe'], "rb") as f:
#     transe = pickle.load(f)

In [2]:
#!g1.1
%load_ext autoreload
%autoreload 2
from model_utils.thswad import THWADModel, load_model_from_transe
from model_utils.trainer import THSWADTrainer
from utils.ds import *

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
#!g1.1
ds_ = DialogDataset(el_single_results['entity_ids'], 'pad', 5)
items2id = {}
for i, (prev, next_id) in enumerate(ds_):
    for item in prev:
        if item not in items2id and item != 'pad':
            items2id[item] = len(items2id)
    if next_id not in items2id and next_id != 'pad':
        items2id[next_id] = len(items2id)
id2items = list(items2id.keys())
ds = DialogDataset(el_single_results['entity_ids'], len(items2id), 5, items2id)
ds[0]

/home/admin/dream-entity-recommendation/utils/ds.py:64: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all(np.array(prev) == pad_item):


(array([8157, 8157, 8157, 8157,    0]), 1)

In [4]:
def optimized_transe_load():
    with open(config['wikidata']['transe'], "rb") as f:
        transe = pickle.load(f)
    return load_model_from_transe(transe, id2items, prev_items_total=5), transe.graph


model, graph = optimized_transe_load()

start ent emb
end ent emb


In [5]:
#!g1.1
wds = load_wiki_dataset(config['wikidata']['dataset']['train'], graph)

In [6]:
#!g1.1
from torch.utils.data import DataLoader

BS = 256

dl = DataLoader(ds, batch_size=BS, shuffle=False)
wdl = DataLoader(wds, batch_size=BS, shuffle=False)
train_loader = TSHWADLoader(wdl, dl)

In [7]:
#!g1.1
# model = load_model_from_transe(transe, id2items, prev_items_total=5)
# model.load('model_checkpoints/model.pt')

In [8]:
#!g1.1
opt = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(opt, 1, gamma=0.9)

trainer = THSWADTrainer(
    margin=.5,
    norm_lambda=.2,
    clipping_max_value=5.,
    model_target=.5,
    kg_lambda=.1,
    model=model,
    optimizer=opt,
    scheduler=scheduler,
    train_data_loader=train_loader,
    val_data_loader=train_loader,
    test_data_loader=train_loader,
    verbose=False,
)

In [9]:
#!g1.1
trainer.train(20)

Training:  16%|█▌        | 25360/160794 [03:27<18:27, 122.29it/s]


KeyboardInterrupt: 

In [24]:
#!g1.1
BS = 32

dl_v = DataLoader(ds, batch_size=BS, shuffle=False)
wdl_v = DataLoader([], batch_size=BS, shuffle=False)
val_loader = TSHWADLoader(wdl, dl)

trainer.test(val_loader)

Validating:  99%|█████████▉| 158870/160794 [5:18:32<03:51,  8.31it/s]


{'accuracy': tensor(0.0033, device='cuda:0')}

In [ ]:
#!g1.1
model.prev_meaner

In [ ]:
#!g1.1
len(model.item2entity)

In [20]:
#!g1.1
! head -n 1000000 dataset/wikidata5m_transductive_train.txt > dataset/wikidata5m_transductive_train_head.txt

In [ ]:
#!g1.1
